Here we want to parse the ENCODE eCLIP beds and subset to only those peaks which pass their "stringent" thresholds. According to https://www.genome.gov/multimedia/slides/encode2016-researchappsusers/vannostrand_eclip.pdf:

"
chr\tstart\tstop\tdataset_label\t1000\tstrand\tlog2(eCLIP fold-enrichment over size-matched input)\t-log10(eCLIP vs size-matched input p-value)\t-1\t-1

Note: p-value is calculated by Fisher’s Exact test (minimum p-value 2.2x10-16), with chi-square test (–log10(p-value) set to 400 if p-value reported == 0)

Our typical ‘stringent’ cutoffs: require -log10(p-value) ≥ 5 and log2(fold-enrichment) ≥ 3 
"

where the top line defines the columns and the final line denotes the thresholds

We'll output each mark into "per-contig" outfiles so that we can use them with bedtools intersect more easily downstream

In [27]:
import glob
import os
import collections
import CGAT.IOTools as IOTools

In [67]:
infiles = glob.glob("../eCLIP_beds/*bed.gz")
infiles = [x for x in infiles if "stringent" not in x]
print(len(infiles))

238


In [21]:
n = 0
with IOTools.openFile(infiles[0]) as inf:
    for line in inf:
        print(line)
        break

chr14_GL000194v1_random	93861	93917	U2AF1_K562_rep02	200	-	3.29346949707327	1.86528752890148	-1	-1



In [58]:
all_peaks = collections.defaultdict(lambda: collections.defaultdict(list))
for infile in infiles:
    mark = os.path.basename(infile).split("_")[0]
    n = 0
    outfile = IOTools.openFile(infile.replace(".bed.gz", "_stringent.bed.gz"), "w")
    contigs_observed = set()
    with IOTools.openFile(infile) as inf:
        for line in inf:
            contig, start, stop, _, __, ___, enrichment, p_value = line.split("\t")[0:8]
            contigs_observed.add(contig)
            enrichment, p_value = map(float, (enrichment, p_value))
            start, stop = map(int, (start, stop))
            enrichments.append(enrichment)
            p_values.append(p_value)

            if enrichment >= 3 and p_value > 5:
                outfile.write(line)
                n+=1
                all_peaks[contig][mark].append((start, stop))
    
    outfile.close()
    
    # reorder peaks by position
    for contig in contigs_observed:
        all_peaks[contig][mark].sort(key=lambda x: x[0])
        outfile = IOTools.openFile(infile.replace(".bed.gz", "_%s_stringent.bed.gz" % contig), "w")
        for peak in all_peaks[contig][mark]:
            start, stop = map(str, peak)
            outfile.write("\t".join((contig, start, stop)) + "\n")
        outfile.close()
        
        
    
    print(infile, n, "\r", end="")
    #break

